<a href="https://colab.research.google.com/github/josegerman/azureml-sdk/blob/main/Full_AzureML_SDK_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 - Installation and setup

In [1]:
# install azureml sdk package
! pip install -q azureml-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.2/136.2 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.4/779.4 kB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [26]:
# import workspace class
from azureml.core import Workspace

In [27]:
# access workspace
ws = Workspace.from_config('/content/config.json')

# Step 2 - Train and serialize ML model

In [4]:
# load dataset
import sklearn
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge

In [5]:
# train model
x, y = load_diabetes(return_X_y=True)
model = Ridge().fit(x, y)

In [7]:
# serialize model into current directory
import joblib
joblib.dump(model, 'regression_model.pkl')

['regression_model.pkl']

# Step 3 - Register ML model

In [8]:
# import class
from azureml.core import Model

In [9]:
model = Model.register(workspace=ws,
                       model_name='regression_model',
                       model_path='./regression_model.pkl',
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.19.1',
                       description='Regression model to predict diabetic progression',
                       tags={'area':'diabetics','type':'regression'})

print('Name:', model.name)
print('Version:', model. version)

Registering model regression_model
Name: regression_model
Version: 1


# Step 4 - Register environment

In [10]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

In [11]:
env = Environment(name='myEnvironment')
env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pip==20.4.4'],
    pip_packages=['azureml-defaults', 'inference-schema[numpy-support]', 'numpy', 'scikit-learn==0.22.1', 'scipy']
)

# Step 5 - Create AKS cluster

In [12]:
from azureml.core.compute import AksCompute, ComputeTarget

In [13]:
cluster_name = 'aksCluster'

if cluster_name not in ws.compute_targets:
  print(cluster_name, 'Cluster does NOT exist. Creating new one...')
  print('Provisioning config for AKS cluster...')

  aks_config = AksCompute.provisioning_configuration(location='eastus',
                                                     vm_size='Standard_D11_v2',
                                                     agent_count=1,
                                                     cluster_purpose='DevTest')
  print('Creating AKS cluster...')
  production_cluster = ComputeTarget.create(workspace=ws,
                                            name=cluster_name,
                                            provisioning_configuration=aks_config)
  production_cluster.wait_for_completion(show_output=True)

else:
  print(cluster_name, 'Cluster already exists. Will use this one...')
  production_cluster = ws.compute_targets[cluster_name]

aksCluster Cluster does NOT exist. Creating new one...
Provisioning config for AKS cluster...
Creating AKS cluster...
InProgress.........................................................................
SucceededProvisioning operation finished, operation "Succeeded"


# Step 6 - Inference configuration

In [14]:
from azureml.core.model import InferenceConfig

In [15]:
inference_config = InferenceConfig(entry_script='score1_entryscript.py', # entry script
                                   source_directory='.',
                                   environment=env)

# Step 7 - Deployment configuration

In [19]:
namespace_name = 'endpointnamespace'
version_name = 'version1'

In [20]:
from azureml.core.webservice import AksEndpoint

In [21]:
endpoint_deployment_config = AksEndpoint.deploy_configuration(tags={'modelVersion':'First', 'Department':'Finace'},
                                                              description='First version',
                                                              namespace=namespace_name,
                                                              version_name=version_name,
                                                              traffic_percentile=10)

# Step 8 - Create end point

In [35]:
# select compute
compute = ComputeTarget(workspace=ws,
                        name='aksCluster')

# endpoint name
endpoint_name = 'myendpoint' # must be all lowercase

In [36]:
endpoint = Model.deploy(workspace=ws,
                        name=endpoint_name,
                        models=[model],
                        inference_config=inference_config,
                        deployment_config=endpoint_deployment_config,
                        deployment_target=compute)
endpoint.wait_for_deployment(True)

<ipython-input-36-c1514cbb9866>:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  endpoint = Model.deploy(workspace=ws,


# Step 9 - Test the web service

In [37]:
import json

In [38]:
test_sample = json.dumps({'data':[
    [1,2,3,4,5,6,7,8,9,10],
    [10,9,8,7,6,5,4,3,2,1]
]})

In [ ]:
test_sample_encoded = bytes(test_sample, encoding='utf8')
prediction = endpoint.run(input_data=test_sample_encoded)
print(prediction)

# Step 10 - Delete endpoint

In [ ]:
endpoint.delete()